# Householder Reflections

Given a vector, $v$, find the matrix that calculates $v$'s reflection such that the first element is the norm of $v$ and the remaining elements are all zero.

## An Arbitrary Reflection
First, given a vector, we will find the matrix which gives projections across a plane orthogonal to our vector.


In [8]:
v = [1.,1.]

2-element Array{Float64,1}:
 1.0
 1.0

The reflector matrix orthogonal to $v$, $H$, is given by

In [9]:
using LinearAlgebra
H = I - 2/(v'*v)*v*v' # Househoulder matrix H, based on Householder vector v

2×2 Array{Float64,2}:
  0.0  -1.0
 -1.0   0.0

The reflection of $v$ itself ($v$ reversed) is given by

In [10]:
H*v

2-element Array{Float64,1}:
 -1.0
 -1.0

$H$ is orthogonal (and symmetric)

In [11]:
H*H'

2×2 Array{Float64,2}:
  1.0  -0.0
 -0.0   1.0

Apply $H$ to an arbitrary $x$.  Then $x$ is reflected across the line orthogonal to $v$.

In [12]:
x = [1;2]
H*x

2-element Array{Float64,1}:
 -2.0
 -1.0

## Reflection onto a standard basis vector
Next, given an arbitrary vector $a$, we want a reflection matrix $H$ which generates a reflection whose first element is the norm of $a$ and whose remaining elements are all zero.

In [6]:
# Householder Reflection
using LinearAlgebra
function householder(a)
    """
    Computes the householder reflection (matrix)
    given a nonzero vector a.
    """
    nrm_a = norm(a,2)
#    display(nrm_a)
    nrm_a == 0 && error("Input vector is zero.")
    
    d = length(a)
    v = copy(a)
    v[1] = v[1] - nrm_a
    H = Matrix(I,d,d) - (2/dot(v,v))*v*v'
    return H
end

householder (generic function with 1 method)

In [15]:
#Example of Householder Reflection

printstyled(
"EXAMPLE: Householder Reflection\n", color=:red)
println("Householder reflection, applied to the vector.")
a = rand(10)
H = householder(a)

display(H*a)
println("Norm of the original vector:
    $(norm(a))")
println("Norm of the projection:
    $(norm(H*a))")

10-element Array{Float64,1}:
  1.7450015906792058    
 -9.475249361165505e-17 
 -2.6105555434377536e-17
 -3.290214780314604e-17 
 -1.5576935900006583e-18
 -5.3998350387461647e-17
 -2.856353916819111e-19 
  2.1836509380215863e-18
 -9.706828168944831e-17 
 -1.8214596497756474e-17

EXAMPLE: Householder Reflection
Householder reflection, applied to the vector.
Norm of the original vector:
    1.745001590679206
Norm of the projection:
    1.7450015906792058


## A Series of Reflections

In [31]:
function house(x)
    # from Golub & Van Loan 2013
    m = length(x)
    σ = x[2:m]'*x[2:m]
    v = [1; x[2:m]]
    
    if σ == 0 
        if x[1] >= 0
            β = 0
        else  
            β = -2
        end
    else
        μ = sqrt(x[1]^2 + σ )
        if x[1] <= 0
            v[1] = x[1] - μ
        else
            v[1] = -σ + v[1]^2
        end
    end
    β = 2*v[1]^2/(σ+v[1]^2)
    v = v/v[1]
    return v, β
end

house (generic function with 1 method)

In [120]:
A = rand(5,5)
house(A[:,1])

([1.0, -0.0786823, -0.739686, -1.22818, -0.92047], 0.511637654188873)

In [196]:
function house_gen(x)
    """ Generate Householder reflection (Moler)
    """
    nu = norm(x)
    if nu == 0
        u = x
        u[1] = sqrt[2]
    else
        u = x/nu
        u[1] = u[1] + sign(u[1])
        u = u/sqrt(abs(u[1]))
    end
    return u
end

house_gen (generic function with 1 method)

In [198]:
display(house_gen(A[:,1]))
#house(A[:,1])

6-element Array{Float64,1}:
 1.273243714530443   
 0.041815553422623115
 0.43209405203377216 
 0.11150814246032831 
 0.4181555342262312  
 0.055754071230164154

([1.0, -0.00153925, -0.0159056, -0.00410467, -0.0153925, -0.00205233], 1.9989738330047406)

In [206]:
function Hvec(u,x)
    return x - u*(u'*x)
end

function house_qr(A)
    """ Householder reflections for QR decomposition.
    % R, the upper triangular factor, and
    % U, the reflector generators for use by house_apply.
    from Moler """

    m,n = size(A)
    U = zeros(m,n)
    R = copy(A)
    for j = 1:min(m,n)
        u = house_gen(R[j:m,j])
        U[j:m,j] = u
        R[j:m,j:n] = Hvec(u,R[j:m,j:n])
        R[j+1:m,j] = repeat([0], m-j)
    end
    return R, U
end

house_qr (generic function with 1 method)

In [210]:
A =[35.     1     6    26    19    24;
     3    32     7    21    23    25;
    31     9     2    22    27    20;
     8    28    33    17    10    15;
    30     5    34    12    14    16;
     4    36    29    13    18    11]

R, U = house_qr(A)
display(R)  # The R of QR
display(U)  # generators for Q

6×6 Array{Float64,2}:
 -56.3471  -16.4693  -30.0459  -39.0969   -38.0321   -38.671      
   0.0     -54.2196  -34.8797  -23.1669   -25.2609   -23.2963     
   0.0       0.0      32.4907   -8.91816  -11.2895    -7.9245     
   0.0       0.0       0.0      -7.62831    3.91136   -7.4339     
   0.0       0.0       0.0       0.0       -3.41967   -6.83935    
   0.0       0.0       0.0       0.0        0.0        8.88178e-16

6×6 Array{Float64,2}:
 1.27324     0.0         0.0        0.0        0.0        0.0    
 0.0418156   1.25683     0.0        0.0        0.0        0.0    
 0.432094    0.0450736  -1.16613    0.0        0.0        0.0    
 0.111508    0.38844     0.455693   1.07392    0.0        0.0    
 0.418156   -0.0108187   0.59417   -0.645475   1.07958    0.0    
 0.0557541   0.517063    0.281857  -0.655788  -0.913516  -1.41421

In [222]:
function house_apply(U,X)
    """ Apply Householder reflections.
     without actually computing Q.  continuing from Moler
    """
    Z = copy(X)
    m,n = size(U)
    for j = n:-1:1
        Z = Hvec(U[:,j],Z)
    end
    return Z
end

house_apply (generic function with 1 method)

In [223]:
display(A)
display(U)
house_apply(U, A)

6×6 Array{Float64,2}:
 35.0   1.0   6.0  26.0  19.0  24.0
  3.0  32.0   7.0  21.0  23.0  25.0
 31.0   9.0   2.0  22.0  27.0  20.0
  8.0  28.0  33.0  17.0  10.0  15.0
 30.0   5.0  34.0  12.0  14.0  16.0
  4.0  36.0  29.0  13.0  18.0  11.0

6×6 Array{Float64,2}:
 1.27324     0.0         0.0        0.0        0.0        0.0    
 0.0418156   1.25683     0.0        0.0        0.0        0.0    
 0.432094    0.0450736  -1.16613    0.0        0.0        0.0    
 0.111508    0.38844     0.455693   1.07392    0.0        0.0    
 0.418156   -0.0108187   0.59417   -0.645475   1.07958    0.0    
 0.0557541   0.517063    0.281857  -0.655788  -0.913516  -1.41421

6×6 Array{Float64,2}:
 -23.4576    7.9998     2.06914  -16.6508    -6.58604  -13.489  
 -38.6814  -31.6133   -34.2437   -34.7041   -37.5524   -38.1671 
 -25.2613   -8.79278    3.30004  -20.4045   -24.0447   -17.4901 
   9.2354  -42.3318   -23.2065   -16.4155   -12.7971   -15.4389 
 -12.92     10.6076   -12.2839    -1.25324    3.24531   -3.67503
  15.4316   12.8756    36.0244     5.83078    5.74418    2.55992

In [224]:

Q = house_apply(U,Matrix{Float64}(I, size(U)))
Q

6×6 Array{Float64,2}:
 -0.62115     0.170232    -0.206992  -0.499813   0.206242   0.5        
 -0.0532414  -0.574021    -0.450017  -0.210639  -0.648677  -2.46331e-16
 -0.550161    0.00112113  -0.446003   0.453726   0.206242  -0.5        
 -0.141977   -0.473293     0.376287  -0.503413   0.332896  -0.5        
 -0.532414    0.0695045    0.628716   0.209555  -0.522022  -2.77556e-16
 -0.0709885  -0.642404     0.137276   0.450126   0.332896   0.5        

In [225]:
Q*Q'

6×6 Array{Float64,2}:
  1.0           2.7924e-16    7.49401e-16  …   2.31315e-16  -5.27356e-16
  2.7924e-16    1.0          -1.99099e-16      5.71261e-17   3.28016e-17
  7.49401e-16  -1.99099e-16   1.0              3.789e-16     5.55112e-16
  4.71845e-16  -1.4472e-16   -1.94289e-16     -1.80238e-16   5.55112e-17
  2.31315e-16   5.71261e-17   3.789e-16        1.0           1.38521e-16
 -5.27356e-16   3.28016e-17   5.55112e-16  …   1.38521e-16   1.0        

In [226]:
Q*R

6×6 Array{Float64,2}:
 35.0   1.0   6.0  26.0  19.0  24.0
  3.0  32.0   7.0  21.0  23.0  25.0
 31.0   9.0   2.0  22.0  27.0  20.0
  8.0  28.0  33.0  17.0  10.0  15.0
 30.0   5.0  34.0  12.0  14.0  16.0
  4.0  36.0  29.0  13.0  18.0  11.0